## How To Clean Data

### Diagnose the Data

We often describe data that is easy to analyze and visualize as “tidy data”. What does it mean to have tidy data?

For data to be tidy, it must have:

- Each variable as a separate column
- Each row as a separate observation

You’ve seen most of the functions we often use to diagnose a dataset for cleaning. Some of the most useful ones are:

- `.head()` — display the first 5 rows of the table
- `.info()` — display a summary of the table
- `.describe()` — display the summary statistics of the table
- `.columns` — display the column names of the table
- `.value_counts()` — display the distinct values for a column


In [122]:
import pandas as pd

df1 = pd.read_csv("./csv/df1.csv")
df2 = pd.read_csv("./csv/df2.csv")

print(df1.head())
print(df2.head())

  Grocery Item  Cake Recipe  Pancake Recipe  Cookie Recipe
0         Eggs            2               3              1
1         Milk            1               2              1
2        Flour            2               1              2
  Grocery Item          Recipe  Number
0         Eggs     Cake Recipe       2
1         Milk     Cake Recipe       1
2        Flour     Cake Recipe       2
3         Eggs  Pancake Recipe       3
4         Milk  Pancake Recipe       2


### Dealing with Multiple Files

Let’s say that we have a ton of files following the filename structure: 'file1.csv', 'file2.csv', 'file3.csv', and so on. The power of pandas is mainly in being able to manipulate large amounts of structured data, so we want to be able to get all of the relevant information into one table so that we can analyze the aggregate data.

We can combine the use of glob, a Python library for working with files, with pandas to organize this data better. glob can open multiple files by using regex matching to get the filenames:

In [123]:
import glob

student_files = glob.glob("./csv/exam*.csv")

df_list = []

for df_file in student_files:
  df = pd.read_csv(df_file)
  df_list.append(df)

students = pd.concat(df_list)

print(students)
print(len(students))

    id         full_name gender_age fractions probability       grade
0    0    Barrett Feragh        M14       76%         72%   9th grade
1    1   Llewellyn Keech        M14       83%         NaN  12th grade
2    2   Llewellyn Keech        M14       83%         NaN  12th grade
3    3      Terrell Geri        M15       80%         86%  11th grade
4    4    Gram Hallewell        M14       67%         78%  10th grade
..  ..               ...        ...       ...         ...         ...
95  95     Donia Stockoe        F18       88%         84%  10th grade
96  96   Korella Spurden        F16       79%         81%   9th grade
97  97     Basilius Sine        M17       66%         85%  10th grade
98  98  Kingston Akaster        M14       71%         75%   9th grade
99  99     Carita Utting        F16       68%         83%  11th grade

[400 rows x 6 columns]
400


### Dealing with Duplicates

Often we see duplicated rows of data in the DataFrames we are working with. This could happen due to errors in data collection or in saving and loading the data.

To check for duplicates, we can use the pandas function `.duplicated()`, which will return a Series telling us which rows are duplicate rows.

We can use the pandas `.drop_duplicates()` function to remove all rows that are duplicates of another row.



In [124]:
duplicates = students.duplicated()
print(duplicates.head())
print(duplicates.value_counts())
students = students.drop_duplicates()
duplicates = students.duplicated()
print(duplicates.head())
print(duplicates.value_counts())

0    False
1    False
2    False
3    False
4    False
dtype: bool
False    400
Name: count, dtype: int64
0    False
1    False
2    False
3    False
4    False
dtype: bool
False    400
Name: count, dtype: int64


### Splitting by Index

In trying to get clean data, we want to make sure each column represents one type of measurement. Often, multiple measurements are recorded in the same column, and we want to separate these out so that we can do individual analysis on each variable.

In [125]:
print(students.columns)
print(students.gender_age.head())

students['gender'] = students.gender_age.str[0]
students['age'] = students.gender_age.str[1:]
print(students.head())
students[['full_name','grade','gender','age']]

Index(['id', 'full_name', 'gender_age', 'fractions', 'probability', 'grade'], dtype='object')
0    M14
1    M14
2    M14
3    M15
4    M14
Name: gender_age, dtype: object
   id        full_name gender_age fractions probability       grade gender age
0   0   Barrett Feragh        M14       76%         72%   9th grade      M  14
1   1  Llewellyn Keech        M14       83%         NaN  12th grade      M  14
2   2  Llewellyn Keech        M14       83%         NaN  12th grade      M  14
3   3     Terrell Geri        M15       80%         86%  11th grade      M  15
4   4   Gram Hallewell        M14       67%         78%  10th grade      M  14


,full_name,grade,gender,age
0,Barrett Feragh,9th grade,M,14
1,Llewellyn Keech,12th grade,M,14
2,Llewellyn Keech,12th grade,M,14
3,Terrell Geri,11th grade,M,15
4,Gram Hallewell,10th grade,M,14
...,...,...,...,...
95,Donia Stockoe,10th grade,F,18
96,Korella Spurden,9th grade,F,16
97,Basilius Sine,10th grade,M,17
98,Kingston Akaster,9th grade,M,14


### Splitting by Character
Let’s say we have a column called “type” with data entries in the format "admin_US" or "user_Kenya". Just like we saw before, this column actually contains two types of data. One seems to be the user type (with values like “admin” or “user”) and one seems to be the country this user is in (with values like “US” or “Kenya”).

We can no longer just split along the first 4 characters because admin and user are of different lengths. Instead, we know that we want to split along the "_". Using that, we can split this column into two separate, cleaner columns:

In [126]:
name_split = students['full_name'].str.split(" ")
students['first_name'] = name_split.str.get(0)
students['last_name'] = name_split.str.get(1)

print(students.head())

   id        full_name gender_age fractions probability       grade gender  \
0   0   Barrett Feragh        M14       76%         72%   9th grade      M   
1   1  Llewellyn Keech        M14       83%         NaN  12th grade      M   
2   2  Llewellyn Keech        M14       83%         NaN  12th grade      M   
3   3     Terrell Geri        M15       80%         86%  11th grade      M   
4   4   Gram Hallewell        M14       67%         78%  10th grade      M   

  age first_name  last_name  
0  14    Barrett     Feragh  
1  14  Llewellyn      Keech  
2  14  Llewellyn      Keech  
3  15    Terrell       Geri  
4  14       Gram  Hallewell  


### Looking at Types

Each column of a DataFrame can hold items of the same data type or dtype. The dtypes that pandas uses are: float, int, bool, datetime, timedelta, category and object. Often, we want to convert between types so that we can do better analysis. If a numerical category like "num_users" is stored as a Series of objects instead of ints, for example, it makes it more difficult to do something like make a line graph of users over time.


In [127]:
students.dtypes


id              int64
full_name      object
gender_age     object
fractions      object
probability    object
grade          object
gender         object
age            object
first_name     object
last_name      object
dtype: object

### String Parsing


In [128]:
students.fractions = students['fractions'].replace('[%,]', '', regex=True)
students.fractions = pd.to_numeric(students['fractions'])

students.head()

,id,full_name,gender_age,fractions,probability,grade,gender,age,first_name,last_name
0,0,Barrett Feragh,M14,76.0,72%,9th grade,M,14,Barrett,Feragh
1,1,Llewellyn Keech,M14,83.0,NaN,12th grade,M,14,Llewellyn,Keech
2,2,Llewellyn Keech,M14,83.0,NaN,12th grade,M,14,Llewellyn,Keech
3,3,Terrell Geri,M15,80.0,86%,11th grade,M,15,Terrell,Geri
4,4,Gram Hallewell,M14,67.0,78%,10th grade,M,14,Gram,Hallewell


In [129]:
print(students.grade.head())

print(students.dtypes)

students.grade = students.grade.str.split('(\d+)', expand=True)[1]
students.grade = pd.to_numeric(students.grade)

avg_grade = students.grade.mean()

print(avg_grade)

0     9th grade
1    12th grade
2    12th grade
3    11th grade
4    10th grade
Name: grade, dtype: object
id               int64
full_name       object
gender_age      object
fractions      float64
probability     object
grade           object
gender          object
age             object
first_name      object
last_name       object
dtype: object
10.6125


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
/var/folders/8q/xpg4bkkd4679dp34plrsh3tw0000gn/T/ipykernel_68891/3186036003.py:5: SyntaxWarning: invalid escape sequence '\d'
  students.grade = students.grade.str.split('(\d+)', expand=True)[1]
